In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('decisiontree').getOrCreate()

In [0]:
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.classification import RandomForestClassifier,GBTClassifier,DecisionTreeClassifier

In [0]:
df = spark.read.format('libsvm').load('/FileStore/tables/sample_libsvm_data.txt')

In [0]:
df.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(692,[127,128,129...|
 1.0|(692,[158,159,160...|
 1.0|(692,[124,125,126...|
 1.0|(692,[152,153,154...|
 1.0|(692,[151,152,153...|
 0.0|(692,[129,130,131...|
 1.0|(692,[158,159,160...|
 1.0|(692,[99,100,101,...|
 0.0|(692,[154,155,156...|
 0.0|(692,[127,128,129...|
 1.0|(692,[154,155,156...|
 0.0|(692,[153,154,155...|
 0.0|(692,[151,152,153...|
 1.0|(692,[129,130,131...|
 0.0|(692,[154,155,156...|
 1.0|(692,[150,151,152...|
 0.0|(692,[124,125,126...|
 0.0|(692,[152,153,154...|
 1.0|(692,[97,98,99,12...|
 1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows

In [0]:
train_df,test_df = df.randomSplit([0.7,0.3])

In [0]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()

In [0]:
dtc_model = dtc.fit(train_df)
rfc_model = rfc.fit(train_df)
gbt_model = gbt.fit(train_df)

In [0]:
dtc_preds = dtc_model.transform(test_df)
rfc_preds = rfc_model.transform(test_df)
gbt_preds = gbt_model.transform(test_df)

In [0]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
label| features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
 0.0|(692,[98,99,100,1...| [0.0,1.0]| [0.0,1.0]| 1.0|
 0.0|(692,[122,123,124...| [31.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[123,124,125...| [31.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[124,125,126...| [31.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[124,125,126...| [31.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[126,127,128...| [31.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[127,128,129...| [31.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[151,152,153...| [31.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[153,154,155...| [31.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[153,154,155...| [31.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[155,156,180...| [31.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[234,235,237...| [31.0,0.0]| [1.0,0.0]| 0.0|
 1.0|(692,[100,101,102...| [0.0,42.0]| [0.0,1.0]| 1.0|
 1.0|(692,[123,124,125...| [0.0,42.0]| [0.0,1.0]| 1.0|
 1.0|(692,[127,128,129...| [0.0,42.0]| [0.0,1.0]| 1.0|
 1.0|(692,[128,129,130...| [0.0,42.0]| [0.0,1.0]| 1.0|
 1.0|(692,[129,130,131...| [31.0,0.0]| [1.0,0.0]| 0.0|
 1.0|(692,[129,130,131...| [0.0,42.0]| [0.0,1.0]| 1.0|
 1.0|(692,[129,130,131...| [0.0,42.0]| [0.0,1.0]| 1.0|
 1.0|(692,[129,130,131...| [0.0,42.0]| [0.0,1.0]| 1.0|
+-----+--------------------+-------------+-----------+----------+
only showing top 20 rows

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [0]:
print('DTC Accuracy:')
acc_eval.evaluate(dtc_preds)

DTC Accuracy:
Out[12]: 0.9230769230769231

In [0]:
print('RFC Accuracy:')
acc_eval.evaluate(rfc_preds)

RFC Accuracy:
Out[13]: 1.0

In [0]:
print('GBT Accuracy:')
acc_eval.evaluate(gbt_preds)

GBT Accuracy:
Out[14]: 0.9230769230769231

In [0]:
dtc_model.featureImportances

Out[18]: SparseVector(692, {99: 0.0538, 406: 0.9462})

In [0]:
rfc_model.featureImportances

Out[19]: SparseVector(692, {127: 0.001, 146: 0.0005, 149: 0.0019, 153: 0.0012, 156: 0.0017, 176: 0.0005, 207: 0.0024, 209: 0.0104, 210: 0.0017, 212: 0.0006, 214: 0.0007, 216: 0.0234, 230: 0.0015, 231: 0.0021, 236: 0.0005, 242: 0.0009, 243: 0.0012, 244: 0.0148, 245: 0.0064, 266: 0.0004, 271: 0.0076, 272: 0.0005, 273: 0.013, 275: 0.0003, 289: 0.007, 290: 0.0061, 295: 0.0012, 297: 0.0005, 299: 0.0072, 314: 0.0017, 317: 0.0161, 320: 0.0028, 321: 0.0008, 322: 0.005, 323: 0.0098, 326: 0.0009, 327: 0.0003, 342: 0.0029, 344: 0.0005, 350: 0.0008, 351: 0.0198, 354: 0.0035, 356: 0.0003, 357: 0.0075, 370: 0.0027, 372: 0.0084, 373: 0.0024, 374: 0.0014, 377: 0.011, 378: 0.0407, 379: 0.0447, 384: 0.0065, 397: 0.0005, 399: 0.0108, 400: 0.0016, 401: 0.0075, 403: 0.0009, 404: 0.0015, 405: 0.0706, 407: 0.0085, 408: 0.0013, 409: 0.0005, 410: 0.0018, 411: 0.0021, 413: 0.0154, 414: 0.0005, 426: 0.006, 427: 0.0065, 430: 0.0006, 431: 0.0017, 432: 0.0067, 433: 0.016, 434: 0.0609, 435: 0.022, 436: 0.0013, 438: 0.0013, 439: 0.0036, 440: 0.0173, 441: 0.01, 453: 0.0021, 454: 0.0195, 455: 0.0068, 456: 0.0006, 457: 0.0059, 460: 0.0025, 461: 0.0366, 462: 0.0296, 464: 0.001, 466: 0.0005, 467: 0.0111, 469: 0.0043, 470: 0.0043, 471: 0.0016, 484: 0.0271, 485: 0.0013, 489: 0.0388, 490: 0.0343, 491: 0.0038, 492: 0.0004, 496: 0.0087, 497: 0.0071, 499: 0.0005, 508: 0.0007, 511: 0.023, 512: 0.0269, 513: 0.005, 515: 0.001, 516: 0.0024, 517: 0.0288, 518: 0.0167, 523: 0.0003, 524: 0.0069, 539: 0.0028, 540: 0.0144, 541: 0.0058, 542: 0.0005, 544: 0.0006, 545: 0.0006, 546: 0.0009, 547: 0.0015, 568: 0.0018, 569: 0.0032, 570: 0.0001, 576: 0.0001, 578: 0.0127, 595: 0.001, 625: 0.0002, 627: 0.0005, 629: 0.0005, 630: 0.0034, 631: 0.0007, 636: 0.0009, 638: 0.0011, 655: 0.0002, 663: 0.0011, 682: 0.0005})

In [0]:
spark = SparkSession.builder.appName('collegedecisiontree').getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/tables/College.csv',inferSchema=True,header=True)

In [0]:
df.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)

In [0]:
df.head(1)

Out[23]: [Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
df.columns

Out[25]: ['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [0]:
assembler = VectorAssembler(inputCols=[
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'
],outputCol='features')

In [0]:
output = assembler.transform(df)

In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer(inputCol='Private',outputCol='Private_index')

In [0]:
output_fixed = indexer.fit(output).transform(output)

In [0]:
output_fixed.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)
-- features: vector (nullable = true)
-- Private_index: double (nullable = false)

In [0]:
final_df = output_fixed.select('features','Private_index')

In [0]:
train_df,test_df = final_df.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

In [0]:
from pyspark.ml import Pipeline

In [0]:
dtc = DecisionTreeClassifier(labelCol='Private_index',featuresCol='features')
rfc = RandomForestClassifier(labelCol='Private_index',featuresCol='features')
gbt = GBTClassifier(labelCol='Private_index',featuresCol='features')

In [0]:
dtc_model = dtc.fit(train_df)
rfc_model = rfc.fit(train_df)
gbt_model = gbt.fit(train_df)

In [0]:
dtc_preds = dtc_model.transform(test_df)
rfc_preds = rfc_model.transform(test_df)
gbt_preds = gbt_model.transform(test_df)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
binary_eval = BinaryClassificationEvaluator(labelCol='Private_index')

In [0]:
print('DTC Accuracy')
print(binary_eval.evaluate(dtc_preds))

DTC Accuracy
0.905096308186196

In [0]:
print('RFC Accuracy')
print(binary_eval.evaluate(rfc_preds))

RFC Accuracy
0.9847512038523275

In [0]:
gbt_preds.printSchema()

root
-- features: vector (nullable = true)
-- Private_index: double (nullable = false)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [0]:
rfc_preds.printSchema()

root
-- features: vector (nullable = true)
-- Private_index: double (nullable = false)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

In [0]:
binary_eval_gbt = BinaryClassificationEvaluator(labelCol='Private_index',rawPredictionCol='prediction')

In [0]:
print('GBT Correct Accuracy')
print(binary_eval_gbt.evaluate(gbt_preds))

print('GBT Incorrect Accuracy')
print(binary_eval.evaluate(gbt_preds))

GBT Correct Accuracy
0.9004815409309792
GBT Incorrect Accuracy
0.9576645264847513

In [0]:
rfc = RandomForestClassifier(numTrees=150,labelCol='Private_index',featuresCol='features')
rfc_model = rfc.fit(train_df)
rfc_preds = rfc_model.transform(test_df)


In [0]:
print('RFC Accuracy')
print(binary_eval.evaluate(rfc_preds))

RFC Accuracy
0.9883627608346709

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
acc_eval = MulticlassClassificationEvaluator(labelCol='Private_index',metricName='accuracy')

In [0]:
rfc_acc = acc_eval.evaluate(rfc_preds)

In [0]:
rfc_acc

Out[55]: 0.938814920720023